In [97]:
from sage.all import *
import struct

DR = RealField(53)

DD = RealField(190)

def double_to_hex(f):
    packed = struct.pack('>d', float(f))
    return '0x' + packed.hex()

def split_double_double(x):
    # Split RR value x into hi + lo (double-double)
    x = RealField(190)(x).exact_rational()
    x_hi = DR(x)  # convert to f64
    x_lo = x - DD(x_hi)
    return (x_lo,x_hi)

def split_triple_double(x):
    # Split RR value x into hi + lo (double-double)
    x = RealField(190)(x).exact_rational()
    x_hi = DR(x)  # convert to f64
    x_mid = DR(x - DD(x_hi))
    x_lo = x - DD(x_hi) - DD(x_mid)
    return (x_lo, x_mid, x_hi)

def print_double_double(mark, x):
    splat = split_double_double(x)
    print(f"{mark}({double_to_hex(splat[0])}, {double_to_hex(splat[1])}),")

def print_triple_double(mark, x):
    splat = split_triple_double(x)
    print(f"{mark}({double_to_hex(splat[0])}, {double_to_hex(splat[1])}, {double_to_hex(splat[2])}),")

def format_dyadic_hex(value):
    l = hex(value)[2:]
    n = 8
    x = [l[i:i + n] for i in range(0, len(l), n)]
    return "0x" + "_".join(x) + "_u128"

def print_dyadic(value):
    (s, m, e) = RealField(128)(value).sign_mantissa_exponent();
    print("DyadicFloat128 {")
    print(f"    sign: DyadicSign::{'Pos' if s >= 0 else 'Neg'},")
    print(f"    exponent: {e},")
    print(f"    mantissa: {format_dyadic_hex(m)},")
    print("},")

In [116]:
from sage.all import *
from mpmath import mp, besseli, taylor

# High precision
mp.prec = 500

# Step 1: Define I0(x) - 1
def shifted_i0(x):
    return besseli(0, x) - mp.mpf(1)

# Step 2: Series of I0(x) - 1 in terms of (x/2)^2
terms = 90
from mpmath import taylor
coeffs = taylor(shifted_i0, 0, terms)

# print(coeffs)

# Step 3: Build series in terms of y = (x/2)^2
R = PolynomialRing(RealField(500), 'y')
y = R.gen()
f = R(0)

for n in range(2, terms, 2):
    k = n // 2
    c = RealField(500)(coeffs[n])
    if n >= 1:
        f += R(c) * y**(k-1) * (4**k)
    else:
        f += R(c) * y**(k-1) * (4**k)

print(f)

x = RealField(107)(3.5593750000924014)

print(f(x**2/4)*(x**2/4) + 1)

# rat_approx = SR(f).power_series(RealField(107)).pade(28, 0)
# numer = rat_approx.numerator()
# denom = rat_approx.denominator()

# # print(numer)
# # print(denom)

# def I0_approx(y):
#     z = (y/2)**2
#     return 1 + z * numer(z) / denom(z)

# # Print result
# print("Rational approximation of I0(x):")
# # pretty_print(I0_approx)

# # Evaluate at some point
# print("I0_approx(1) =", I0_approx(-8.5))
# print("I0_real(1) =", besseli(0, -0.5))

# print("Numerator polynomial coefficients:")

# coeffs = numer.coefficients(sparse=False) 

# for i, c in enumerate(coeffs):
    # print("")
    # print_double_double("")
    # print("f64::from_bits(" + double_to_hex(c) + "),")

1.41514618630778449023136504789800043384960096405754551949596226075670317035088121461113881167744441986718863686989614761107450809403356362836400042839e-109*y^43 + 2.73972301669187077308792273273052883993282746641540812574418293682497733779930603148716473940753239686287720098011894177504024767004897918451270482937e-106*y^42 + 5.06574785786326905943956913281874782503579798540208962450099425018938309759091685221976760316452740179945994461223992334204941794192056251216399122950e-103*y^41 + 8.93597922127080662085139995029227116336314764624928609761975385733407178415037732731567005198222633677424734229599122477537517324954787227145728052886e-100*y^40 + 1.50213810709562259296512033164413078256134511933450499300988062341785746691567842872176413573821224721175097823995612488474056662324899732883196885690e-96*y^39 + 2.40342097135299614874419253063060925209815219093520798881580899746857194706508548595482261718113959553880156518392979981558490659719839572613115017104e-93*y^38 + 3.6556032974279071

In [4]:
from mpmath import mp, besseli

# Setup precision
mp.dps = 100

# Define the interval for y (example: from 0 to 3)
a = 0.0
b = 7.5

# Number of test points
N = 1000

# Use your RealField 300 for evaluation
RF = RealField(107)

# Prepare to store max relative error
max_rel_error = 0

for i in range(N+1):
    y_val = a + (b - a) * i / N
    y_rf = RF(y_val)
    
    approx_val = I0_approx(y_rf)
    true_val = mp.besseli(0, y_val)
    
    # Compute relative error
    rel_err = abs(approx_val - true_val) / abs(true_val)
    
    if rel_err > max_rel_error:
        max_rel_error = rel_err

print(f"Maximum relative error on [{a}, {b}] is about {-RealField(70)(max_rel_error).log2()}")

Maximum relative error on [0.0, 7.5] is about 105.43635783664410671


In [5]:
from mpmath import mp, besseli, nprint, taylor, chebyfit

# High precision
mp.prec = 1000

# Define the function to expand
def f(x):
    return mp.sqrt(x) * mp.exp(-x) * besseli(0, x)

def f_v(v):
    x = mp.mpf(1) / v
    return f(x)

# Generate coefficients of series in u = 1/x
terms = 80
coeffs = taylor(lambda u: f_v(mp.mpf(u)), mp.mpf('1e-70'), terms)

# print(coeffs)

def f_z(v):
    x = mp.mpf(1) / v
    return f(x)

# coeffs = chebyfit(lambda u: f_z(mp.mpf(u)), [1/9.5, 1/709], terms)
# coeffs = list(reversed(coeffs))
# print(cheby)

# print(coeffs)

# for n in range(terms):
#     u = mp.mpf('1e-30')  # Very small u ≈ 1/x
#     deriv = mp.diff(lambda u: f(1/mp.mpf(u)), u, n)
#     coeff = deriv / mp.fac(n)
#     coeffs.append(coeff)
    # print(f"u^{n}: {coeff}")

x = var('x')
u = x
# shifted_poly = sum(RealField(120)(coeffs[k]) * (u - u0)**k for k in range(terms))
coeffs = [QQ(RealField(120)(coeffs[k]).exact_rational()) for k in range(terms)]
# print(shifted_poly)

# Construct polynomial approximation in 1/x
# P_ring = PolynomialRing(RR, 'u')
taylor_poly = sum(c * u**k for k, c in enumerate(coeffs))

# print(taylor_poly)

pade_approximant = SR(taylor_poly).power_series(RealField(120)).pade(26, 0)

expr_in_x = pade_approximant.subs(u = 1/x)

numer_asympt = expr_in_x.numerator()
denom_asympt = expr_in_x.denominator()

# print(numer_asympt)

# print(denom_asympt)

print(float(f(10.1)))
print(float(numer_asympt(RealField(53)(1/10.1))/denom_asympt(RealField(53)(1/10.1))))
print(taylor_poly(x=1/RealField(53)(1/10.1)))
print(expr_in_x(x=1/10.1))

coeffs = numer_asympt.coefficients(sparse=False) 

print("coeffs")

for i, c in enumerate(coeffs):
    # print_double_double("", c)
    print(RealField(53)(c))
    # print("f64::from_bits(" + double_to_hex(c) + "),")

0.404188505662345
0.40418850661261374
5.21818307638252e169
0.40418850661261374
coeffs
0.398942280401433
0.0498677850501791
0.0280506290907257
0.0292194053028393
0.0447422143699727
0.0906029840991947
0.228395022416720
0.689263549793316
2.42319216724213
9.72642411573575
43.8904888222576
219.951199666086
1212.02275649333
7283.79060392626
47409.6727701986
332262.789997809
2.49456672803042e6
1.99748762266554e7
1.69925162344811e8
1.53044438980294e9
1.45487869805642e10
1.45574469728145e11
1.52935905981443e12
1.68312613919795e13
1.93647168827514e14
2.32473426177431e15
2.90703548792066e16


In [7]:
from mpmath import mp, besseli

# Setup precision
mp.prec = 150

# Define the interval for y (example: from 0 to 3)
a = 9.0
b = 13.0

# Number of test points
N = 1000

# Use your RealField 300 for evaluation
RF = RealField(107)

# Prepare to store max relative error
max_rel_error = 0

def I0_approx_asympt(y):
    return numer_asympt(x=1/y) / denom_asympt(x=1/y) * RealField(53)(y).exp() / RealField(53)(y).sqrt()

for i in range(N+1):
    y_val = a + (b - a) * i / N
    y_rf = RF(y_val)
    
    approx_val = I0_approx_asympt(y_rf)
    true_val = mp.besseli(0, y_val)
    
    # Compute relative error
    rel_err = abs(approx_val - true_val) / abs(true_val)
    
    if rel_err > max_rel_error:
        max_rel_error = rel_err

print(f"Maximum relative error on [{a}, {b}] is about {-RealField(107)(max_rel_error).log2()}")

Maximum relative error on [9.0, 13.0] is about 24.44990328711527391079110708667


In [6]:
from mpmath import mp, besseli, nprint

# High precision
mp.prec = 150
print(besseli(0, 16.225000000016742))

1110969.1973027737067063330973042313994430942


In [7]:
from mpmath import mp, besseli, chebyfit
from sage.all import *

# Set high precision in mpmath
mp.prec = 53

# Step 1: Define the function
def f(x):
    return mp.sqrt(x) * mp.exp(-x) * besseli(0, x)

coeffs = chebyfit(lambda u: f(mp.mpf(u)), [8, 709.782712893384], 27)
print(coeffs)

for i, c in enumerate(reversed(coeffs)):
    # print_double_double("", c)
    # print(c)
    print("f64::from_bits(" + double_to_hex(c) + "),")


[mpf('5.1033286636335007e-65'), mpf('-4.9448046485473206e-61'), mpf('2.2644503491559604e-57'), mpf('-6.5181879689808576e-54'), mpf('1.323100797534861e-50'), mpf('-2.0141049641424839e-47'), mpf('2.387233204663285e-44'), mpf('-2.2578753029007174e-41'), mpf('1.7327243128674407e-38'), mpf('-1.0912375418023538e-35'), mpf('5.6826614926496948e-33'), mpf('-2.4583026152180637e-30'), mpf('8.8538394385740462e-28'), mpf('-2.6551324549089779e-25'), mpf('6.6174596019044181e-23'), mpf('-1.3655130349836739e-20'), mpf('2.3193546998441735e-18'), mpf('-3.2166452699116028e-16'), mpf('3.6040093138654318e-14'), mpf('-3.217796031208781e-12'), mpf('2.249536548765481e-10'), mpf('-1.2039009184205734e-8'), mpf('4.789637519369099e-7'), mpf('-1.3621786829552683e-5'), mpf('0.00026227316601194486'), mpf('-0.0031505266377368203'), mpf('0.4194041232607285')]
f64::from_bits(0x3fdad784644d97aa),
f64::from_bits(0xbf69cf221bf9e8bb),
f64::from_bits(0x3f313036abaf0acd),
f64::from_bits(0xbeec912420536339),
f64::from_bits(0x3

In [133]:

values = [
' 1 ',
' 0.2500000000000000000000000000000000011931267660846218205882675852805793629859455401236688343313829818725935183465480804443359375 ',
' 2.777777777777777777777777777777756890386005395694874028299116305215932904636758784644272324282354702518205158412456512451171875e-2 ',
' 1.736111111111111111111111111115594952902706368755437453406891564355213092676302975794308427852363774235300297732464969158172607421875e-3 ',
' 6.944444444444444444444444436146181435479985262368905587792601529956808196309433808698500354317402483150800662770052440464496612548828125e-5 ',
' 1.929012345679012345679013259308116487792902355964246198116470122375580035918219442969806276237817142804598091743173426948487758636474609375e-6 ',
' 3.93675988914084152179324796587784618572481147917019345059853684660129146625534134199725622321731799304993237864636057565803639590740203857421875e-8 ',
' 6.151187326782564878125489185605986649304263219749565476840275810438348486856600146755594895435750027855455711323084955211015767417848110198974609375e-10 ',
' 7.594058428126623192966648473696939406417920348573727185988244204775353078631950745366388799693611039604746349319236431441026979882735759019851684570312e-12 ',
' 7.594058428126652018203505764892937718345163447561286254724014817870316782511497695854438449762318863189935340518685498445439208126117591746151447296143e-14 ',
' 6.276081345553846592212421286556347590506158657670832352887044239131137284804648014014778869804339876344889741348082468769264963270870794076472520828247e-16 ',
' 4.358389824033745664426510849196931363459398187018846349951141271779772520462521350343200318494290415747632195032052833409328112396208254608609422575682e-18 ',
' 2.578928817679451757539167133589899149795819245308517922355376453934338177809274767619684004064355362025226850161112856329298764074264838086492090951651e-20 ',
' 1.3157850298867677281364021607026590280379272938835195054629189193674475035251824799558128334849587459869628252912816566013168967556683751352064604134284e-22 ',
' 5.845605948155347594116714715841388888089861430838434876446409974719461917359419505995936368698739262947288771004140449445388828133525056617155954419296e-25 ',
' 2.348059170575590826138630127110499188650726083020995358760366420690343291357658707807958558712884713163406954415382143371942121200648903332597843126792e-27 ',
]


for i, c in enumerate(values):
    c = RealField(1000)(c.replace(" ", ""))
    # print_triple_double("", c)
    # print_double_double("", c)
    # print(RealField(53)(c))
    # print("f64::from_bits(" + double_to_hex(c) + "),")
    # print(double_to_hex(c) + ",")
    print_dyadic(c)

DyadicFloat128 {
    sign: DyadicSign::Pos,
    exponent: -127,
    mantissa: 0x80000000_00000000_00000000_00000000_u128,
},
DyadicFloat128 {
    sign: DyadicSign::Pos,
    exponent: -129,
    mantissa: 0x80000000_00000000_00000000_0000032c_u128,
},
DyadicFloat128 {
    sign: DyadicSign::Pos,
    exponent: -133,
    mantissa: 0xe38e38e3_8e38e38e_38e38e38_e36b8460_u128,
},
DyadicFloat128 {
    sign: DyadicSign::Pos,
    exponent: -137,
    mantissa: 0xe38e38e3_8e38e38e_38e38e39_121e517c_u128,
},
DyadicFloat128 {
    sign: DyadicSign::Pos,
    exponent: -141,
    mantissa: 0x91a2b3c4_d5e6f809_1a2b3c17_8294c79a_u128,
},
DyadicFloat128 {
    sign: DyadicSign::Pos,
    exponent: -146,
    mantissa: 0x81742e04_4c5b8724_890a46d5_e25d127c_u128,
},
DyadicFloat128 {
    sign: DyadicSign::Pos,
    exponent: -152,
    mantissa: 0xa91521fb_2a434d3f_641a8661_7a3e4d3e_u128,
},
DyadicFloat128 {
    sign: DyadicSign::Pos,
    exponent: -158,
    mantissa: 0xa91521fb_2a434d40_082851bb_c7dadec6_u128,
},


In [9]:
# Use arbitrary precision and special functions

# Set the target overflow limit for f64
f64_max = RealField(400)(2)**1024 * (RealField(400)(1) - RealField(400)(2**-53))  # == f64::MAX

# Use high precision for computations
R = RealField(400)

# Define the function to compare I0(x) with f64::MAX
def bessel_diff(x):
    return besseli(1, R(x)) - f64_max

# Use binary search to find the cutoff value where I0(x) ≈ f64::MAX
def find_cutoff(x_lo=700, x_hi=720, tol=1e-12):
    while x_hi - x_lo > tol:
        x_mid = (x_lo + x_hi) / 2
        if bessel_diff(x_mid) > 0:
            x_hi = x_mid
        else:
            x_lo = x_mid
    return (x_lo + x_hi) / 2

cutoff = find_cutoff()
print("Cutoff x where I0(x) ≈ f64::MAX is:", cutoff)
print("I0(cutoff) =", bessel_I(1, cutoff))


Cutoff x where I0(x) ≈ f64::MAX is: 713.9876098185423
I0(cutoff) = inf


In [10]:
from sage.all import *
from mpmath import mp, taylor

mp.prec = 500

print(float(mp.besseli(1, -0.00019641107078895875)))

def f(x):
    return mp.besselj(1, x) 

terms = 30
coeffs = taylor(f, mp.mpf(0), terms)

prec_bits = 350
RR = RealField(prec_bits)
L = LaurentSeriesRing(RealField(500), 'x', default_prec = 500)
x = L.gen()

series = L(0)
for n, c in enumerate(coeffs):
    if c != 0:
        series += RR(c) * x**(n)

# Optional: print the series
print(series(subs=x).truncate(20))


-9.820553586804254e-05
0.500000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000*x + 0.000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000*x^2 - 0.0625000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000*x^3 + 0.000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000*x^4 + 0.00260416666666666666666666666666666666666666666666666666666666666666666666666666666666666666666666666666667*x^5 + 0.000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000*x^6 - 0.0000542534722222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222*x^7 + 0.000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000*x^8 + 6.7816840277777777777777777777777777777777777777777777777777

In [11]:
from mpmath import mp, taylor

import numpy as np

print(mp.besselk(1, 0.00025238024))
print(np.float32((mp.besselk(1, 0.00025238024))))

3962.27421187039566410098264693680058081741191737161137607427821099428075085462880283537778787967226526477089756964874964578129485634023301450824426594
3962.2742


In [142]:
R = PolynomialRing(RealField(107), 'z')
z = R.gen()
x = 1.2533141373155002419035727829162900815285534392705444 + 10.635433183254121541220236246429533099101008352895388*z + 20.174562190882996054154135155495106754234956241084497 *z**2 - 2.0124074566655218437700226406848876215293993602012009 *z**3 + 0.85730507521885754875680381278184681499877612573590721* z**4 - 0.64425895157008493270742725518393343576294899792536059* z**5 + 0.67053819037846809445676366792093172873542726698912348* z**6 - 0.85413500472728190956077165616865651403453673819423044* z**7 + 1.2239367955534735097474473321330242560821280353852232 *z**8 -  1.8447712859556259875259980609771210851930600191167760 *z**9 +  2.7636535917052952197873013830688588947414821008593453 *z**10 - 3.9240434118934010665799546313458516292033791432973163 *z**11 + 5.0777172436286910307426635134078788938504524879473308 *z**12 - 5.7974913539397550997755262645955779659647166218306631 *z**13 + 5.6807429630004755423328685794483075919280929751669777 *z**14 - 4.6555189128490244724632393339598798027214431882326775 *z**15 +  3.1064686409479559680372158221454582538304036291933162 *z**16 -  1.6349690402036198588459906106055152603676933476862251 *z**17 + 0.65042065733394472693321209960702682292340646867511452* z**18 - 0.18325191162927484578925149742335792209585086825186654* z**19 + 0.032510724379786101549632646211334498315782854292177549 *z**20 - 0.0027260051898979127045433464129385581691757697459122891* z**21

P = x.polynomial(z)

# Extract coefficients as a list (ordered from constant term up)
coeffs = [
' 8.3333333333333333333333333333333333333823122646175953128320306890509269032424443981988369800628646544282673858106136322021484375e-2 ',
' 6.944444444444444444444444444444444125326299239442538852306714712773339165790393901994226148743916127159536699764430522918701171875e-3 ',
' 3.472222222222222222222222222222476211452418484169330936887034839181431732752210384686418422235665293129613928613252937793731689453125e-4 ',
' 1.157407407407407407407407407330412729836518807467034563160242695565880749475684553216848506922349339165467796419761725701391696929931640625e-5 ',
' 2.755731922398589065255732043201284947574227442771539828835347520920539873860906711533905581211893764427556252627482535899616777896881103515625e-7 ',
' 4.9209498614260519022422649283844868216916823551124239735402447643620202869092822405279728378792887996651637949607760447179316543042659759521484375e-9 ',
' 6.8346525853139609754060125746990924528463853177192157630769388616099143631021127021754128308339326394220338174723128332743726787157356739044189453125e-11 ',
' 7.594058428126623276921412575262925513776779054911954584470510352795408656369577841245438082950533797962198576580929859636270862210949417203664779663086e-13 ',
' 6.903689480115120650430075169831018575022775116809869810661238297080609828376563959096559107348472246949140101461271927692220362615671547246165573596954e-15 ',
' 5.230067787964185980191270128879402432961675885552171840039777965433311982673981314172703952941902659869073084544294614957847533442958365412778221070766e-17 ',
' 3.35260755666347416961291689230965397927232288019794089528233283408119000616906189567896609286562614144566522514953186778144665928722001346784509223653e-19 ',
' 1.842092034104411540084974383516279434668290790590397103768536589436760633434074576150661176236925399989341757216182311261925262803719818838210642297781e-21 ',
' 8.771889506602143979653119769544117090052061574833143176109209988412536414564321494524940009957483224124802089369465367664498594010251132343028435123244e-24 ',
' 3.653820385687507228817372107150669747110807673055506564426298528977019600275564224145273464963054125053392187099906074111832916005905966524221380187232e-26 ',
' 1.376988837202016121425023366084915407399420856616616934043081519223030242745791660960575026674678767328794391873790686661328290491513886782674958231742e-28 ',
]

# Print nicely
for c in coeffs:
    # print("f64::from_bits(" + double_to_hex(c) + "),")
    # print_double_double("", c)
    # print_triple_double("", c)
    print_dyadic(c)
    # print(c)

print("--")
print("f64::from_bits(" + double_to_hex(RealField(107)('1')) + "),")
print("f64::from_bits(" + double_to_hex(RealField(107)('8.6108479343670076529219106105176467832092478822742928')) + "),")
print("f64::from_bits(" + double_to_hex(RealField(107)('17.103015184855466847985374789136626320766401206703402')) + "),")

DyadicFloat128 {
    sign: DyadicSign::Pos,
    exponent: -131,
    mantissa: 0xaaaaaaaa_aaaaaaaa_aaaaaaaa_aaaaaaac_u128,
},
DyadicFloat128 {
    sign: DyadicSign::Pos,
    exponent: -135,
    mantissa: 0xe38e38e3_8e38e38e_38e38e38_e38e0298_u128,
},
DyadicFloat128 {
    sign: DyadicSign::Pos,
    exponent: -139,
    mantissa: 0xb60b60b6_0b60b60b_60b60b60_b7197728_u128,
},
DyadicFloat128 {
    sign: DyadicSign::Pos,
    exponent: -144,
    mantissa: 0xc22e4506_72894ab6_cd8efb0d_d3d0066a_u128,
},
DyadicFloat128 {
    sign: DyadicSign::Pos,
    exponent: -149,
    mantissa: 0x93f27dbb_c4fae397_780b71cc_6113c272_u128,
},
DyadicFloat128 {
    sign: DyadicSign::Pos,
    exponent: -155,
    mantissa: 0xa91521fb_2a434d3f_648ce9bb_6361231e_u128,
},
DyadicFloat128 {
    sign: DyadicSign::Pos,
    exponent: -161,
    mantissa: 0x964bac6d_7ae67d8d_caeaeaca_95245b64_u128,
},
DyadicFloat128 {
    sign: DyadicSign::Pos,
    exponent: -168,
    mantissa: 0xd5c0f53a_fe6fa144_b6920c03_c832a616_u128,
},


In [149]:
# K1 series

from mpmath import mp, euler

V = PowerSeriesRing(RealField(150), 'z')

z = V.gen()

mp.prec = 150

R = RealField(200)

euler_gamma = R(euler)

lg = R(2).log()
lg4 = R(4).log()
lg8 = R(8).log()

expr = (z**17 * (R(-6989) + R(2520) * euler_gamma - R(2520) * lg))/R('4832746593583104000') \
        + (z**15 * (R(-1487)+R(560) * euler_gamma-R(560) *lg))/R('3728971137024000')\
        +(z**13 * (R(-353)+R(140) * euler_gamma-R(140) * lg))/R('4161798144000')\
        +(z**9 * (R(-131)+R(60) * euler_gamma-R(60) * lg))/R('88473600')\
        +(z**11 * (R(-71)+R(30) * euler_gamma - R(30) *lg))/R('5308416000')\
        +(z**7 * (R(-47)+R(24) * euler_gamma - R(24) * lg))/R(442368)\
        +R(1/R(64)) * z**3 * (R(-5)+R(4) * euler_gamma-R(4) * lg)\
        +R(1/4) * z * (R(-1)+R(2) * euler_gamma-2 * lg)\
        +(z**5 * (R(-5)+R(3) *euler_gamma-3 * lg))/R(1152)\
        + (z**21 * (R(-82451)+R(27720) * euler_gamma-R(27720) * lg))/R('8420577664659200409600000')\
        + (z**23 * (R(-42433)+R('13860') * euler_gamma-R('13860') * lg))/R('2223032503470028908134400000')\
        + (z**25 * (R('-1132133')+R('360360') * euler_gamma-R('360360')* lg))/R('36066479336297749005572505600000')\
        + (z**27 * (R('-1158863')+R('360360') * euler_gamma - R('360360') * lg))/R('26256396956824761276056784076800000')\
        + (z**29 * (R('-236749')+R('72072') * euler_gamma-R('72072') * lg))/R('4411074688746559894377539724902400000')\
        + (z**31 * (R('-4828073') + R('1441440') * euler_gamma - R('1441440') * lg))/R('84692634023933949972048762718126080000000')
# print(expr)

coeffs = expr.list()  # This gives all coefficients up to the current precision
for i, c in enumerate(coeffs):
    if c != 0:
        # print_dyadic(c)
        print_double_double("", c)
        # print("f64::from_bits(" + double_to_hex(c) + "),")
        # print(f"z^{i}: {c}")

expr2 = -euler_gamma + lg + R(1/4) *  z**2 *(R(1) - euler_gamma + lg) + (\
 z**12 *(R(49) - R(20) * euler_gamma + R(20) *lg))/R('42467328000') + (\
 z**10 *(R(137) - R(60) *euler_gamma + R(60) *lg))/R('884736000') + (\
 z**14 *(R(363) - R(140)* euler_gamma + R(140) *lg))/R('58265174016000') + (\
 z**16 *(R(761) - R(280) *euler_gamma + R(280) *lg))/R('29831769096192000') + (\
 z**18 *(R(7129) - R(2520) *euler_gamma + R(2520) *lg))/R('86989438684495872000') +\
 R(1/128)* z**4 *(R(3) - R(2) * euler_gamma + lg4) + (\
 z**6 *(R(11) - R(6) * euler_gamma + R(64).log()))/R(13824) + (\
 z**8 *(R(25) - R(12) * euler_gamma + R(4096).log()))/R(1769472)
# print(expr2)
print("k0 small")

coeffs = expr2.list()  # This gives all coefficients up to the current precision
for i, c in enumerate(coeffs):
    if c != 0:
        # print_double_double("", c)
        print_dyadic(c)
        # print("f64::from_bits(" + double_to_hex(c) + "),")
        # print(f"z^{i}: {c}")

R = RealField(200)

expr3 = R('1.2533141373155002546825431555831963990977483071423532')+R('11.581464036148641246581323440149428509260059307098616') * z+\
R('26.789405266295715810558337396548064861031194327368438')* z**2 + R('7.3649417349531436967188010018370139100180573487266502') * z**3 \
- R('1.7096604692508150493340167930493663713673704318166940') * z**4+R('1.0804255276094591411415638947116978390123141265771834') *z**5\
+ R('-1.0469297880831626083930679006829252269737874877936028') * z**6+R('1.3002369515375729922365615519495020332408157597613544') * z**7 \
+ R('-1.8716868006869984282123524738168938690121343397007868') * z**8+R('2.9031694730522518237131899238517881330616803775751557') * z**9\
+R('-4.5732446918809058964898044936311043412823198783007045') * z**10+R('6.9647249644245100963438249211100614638042076172726253') * z**11\
+R('-9.8478668366535772967541646404708100660109176278818808') * z**12+R('12.509168035842933091158686315206698671310735894796194')*  z ** 13\
+ R('-13.889426789762708072171011520013585808954445304261549') * z**14+R('13.160468639020739474423102361556011097677163142977284') * z**15\
+ R('-10.397245062168499847721119740813368446763822759405222') * z**16+R('6.6791470271098701770545337257192390539281302865779416')* z**17\
+ R('-3.3836343614489495447896172378322198542839494659295979') * z**18+R('1.2963434524119789098189498423127080263478530079657214') * z**19\
+ R('-0.35208923494132138684480968071673794226763563154951347') * z**20+R('0.060290645206928280777309624029614864219397241979017862') * z**21
+ R('-0.0048861904873423508267690490483453010423241530137713923') * z**22

print("s")

coeffs = expr3.list()  # This gives all coefficients up to the current precision
for i, c in enumerate(coeffs):
    if c != 0:
        print("f64::from_bits(" + double_to_hex(c) + "),")
        # print(f"z^{i}: {c}")

num_expr = R('1') + R('8.8656713459366445640627099502526076307634900011454201') * z + R('18.167413600340225608199066162434028713917004023936959') * z**2

print("s")

coeffs = num_expr.list()  # This gives all coefficients up to the current precision
for i, c in enumerate(coeffs):
    if c != 0:
        print("f64::from_bits(" + double_to_hex(c) + "),")
        # print(f"z^{i}: {c}")

expr5 = R('1.2533141373155008574274621176544090441381452527447420') + R('9.6076379479355559677000707539871221691292104908849198') *z+ R('18.406096659958275387699898145740773014596806776002146')* z**2+ R('4.7300780306398066569023595697768560808449485831009526') * z**3 - R('1.0163754739336227088791404523051013401612222070368604') * z**4 + R('0.58130377839410367882508833965831895831753991773665285') * z**5 - R('0.49232098929036983678046222511723773395343872206074951') * z**6 + R('0.50845889394645196319680354438342957369507316973394789') * z**7 - R('0.57007995718786118274924078452898987222644530044713499') * z**8 + R('0.63665165434076501178161838064495936354739233673543394') * z**9 -R('0.66155559914107090752230117604161227614823201013482179')  *z**10 + R('0.60514236339380552468218358490833810798530137756542675') * z**11 - R('0.46461779005424436911181683303777110485398428882497891') * z**12 + R('0.28604062619504129802564994120724513947211384010839769') * z**13 - R('0.13409133022404639058502814678010866732272117796110097') * z**14 + R('0.044588941897052332965529317763488333331500636452569986') * z**15 - R('0.0093174632995096483384846461968063098276613230731306592') * z**16 + R('0.00091636018782473506248122477549734767637011116719735765') * z**17
print("s17")

coeffs = expr5.list()  # This gives all coefficients up to the current precision
for i, c in enumerate(coeffs):
    if c != 0:
        print("f64::from_bits(" + double_to_hex(c) + "),")
        # print(f"z^{i}: {c}")

expr6 = R('1') + R('7.2907859844421536422466214803077115972643852182573845') * z + R('12.069083105317276679173254499363797483894674077685109') * z**2

print("s6")

coeffs = expr6.list()  # This gives all coefficients up to the current precision
for i, c in enumerate(coeffs):
    if c != 0:
        print("f64::from_bits(" + double_to_hex(c) + "),")
        # print(f"z^{i}: {c}")

expr7 = R('1.2533141373155058832020565569172201040757811539960052') +\
R('8.7651973412137614264861540662854425414839427691011145') * z +\
R('15.301333383136817613847672052255163247878215452807969') * z**2 +\
R('3.7829834554855261433752777441348702648227134959720771') * z**3 -\
R('0.77593806509812267632818079807631581270345383814683018') * z**4 +\
R('0.41676154202785490700268735525325363401025859665280452') * z**5 -\
R('0.32321613121063810175040498613776183097624003643340030') * z**6 +\
R('0.29515078528633240747470434929670937103337640937811351') * z**7 -\
R('0.28002654532055288374464773714587315634778248247707827') * z**8 +\
R('0.25154585259411234638788530725428550333092636929147498') * z**9 -\
R('0.19864146379123211745095983722287988679278308045597601') * z**10 +\
R('0.12933371606491300620629445862636806968339959524282497') * z**11 -\
R('0.065132103886990260625436025198517761428939721122027237') * z**12 +\
R('0.023450256888942946510235248277178016489682020971444765') * z**13 -\
R('0.0053217258841870574666868147901043559815181047238250563') * z**14 +\
R('0.00056843502842415578402336351315730594994654329942957717') * z**15
print("s7")
coeffs = expr7.list()  # This gives all coefficients up to the current precision
for i, c in enumerate(coeffs):
    if c != 0:
        print("f64::from_bits(" + double_to_hex(c) + "),")
        # print(f"z^{i}: {c}")

expr8 = R('1') + R('6.6186156309497610058422408472475779935941069007337244') * z + R('9.8439043035970074081829694480068799884816470174605034') * z**2

print("s8")
coeffs = expr8.list()  # This gives all coefficients up to the current precision
for i, c in enumerate(coeffs):
    if c != 0:
        print("f64::from_bits(" + double_to_hex(c) + "),")
        # print(f"z^{i}: {c}")


(0xbc7037c12ba0b815, 0xbfd3b5b6028a83d6),
(0xbc4037c12ba0b815, 0xbfb5dadb014541eb),
(0x3c1e264dd50350dd, 0xbf7303ae729ff30f),
(0x3bbeb7d012892972, 0xbf1d802af7a5dbc8),
(0x3b188cf6afd16ea8, 0xbeba291822473f2f),
(0xbaefd1fdc38805b2, 0xbe4e212a001aa46f),
(0x3a74c549ad19d196, 0xbdd8630abd83ba61),
(0x39fc11cc02c28333, 0xbd5d49398f1e78b6),
(0x396ee0f0496b5608, 0xbcdb24176f948c55),
(0xb8652b82b72eb179, 0xbbc80559d1876ef2),
(0x37de784e3dddab40, 0xbb37f31cacac2b15),
(0xb73338c4472cd7bf, 0xbaa4258454bb73f4),
(0xb6a61d9412abaadf, 0xba0cfbb72084e258),
(0xb615465b8a66cf2b, 0xb9720837916e4500),
(0xb5601da026853697, 0xb8d399792b0b3231),
k0 small
DyadicFloat128 {
    sign: DyadicSign::Pos,
    exponent: -131,
    mantissa: 0xed6d80a2_a0f59037_c12ba0b8_15015ac5_u128,
},
DyadicFloat128 {
    sign: DyadicSign::Pos,
    exponent: -129,
    mantissa: 0x8ed6d80a_2a0f5903_7c12ba0b_815015ac_u128,
},
DyadicFloat128 {
    sign: DyadicSign::Pos,
    exponent: -133,
    mantissa: 0xced6d80a_2a0f5903_7c12ba0b_8150

In [159]:
expr_k0 = R('1.2533141373154948366080041951960582911146035121315740') +     \
R('8.2245773899079526320913715531861187489858334082181407') * z +  \
R('11.659225568069422689144269600379982654892649199544085') * z**2 -\
R('1.0798339813009586546858974989485617551600570757328948') * z**3 +\
R('0.41395488588116326367650677076555338420712511722896507') * z**4 - \
R('0.26882235446941534398867801042393895684894242532868239') * z**5 + \
R('0.22818681551596479442117213053367341432677188128277528') * z**6 - \
R('0.21895425336695130664551164207288095208476881669620451') * z**7 + \
R('0.21372536171853447391431857503665912965806458885073050') * z**8 - \
R('0.19513656746685727368530532818280649680550011563081530') * z**9 + \
R('0.15547852946970707334322463350358673750505169909153039') * z**10 -\
R('0.10167975086229470304199304100489815474211563951174081') * z**11 +\
R('0.051289281594646311701768857628948719930473359198687553') * z**12 -\
R('0.018464229923427865340456934685352521443259755705199029') * z**13 +\
R('0.0041851949014646505623635614938738768135156088980293181') * z**14 -\
R('0.00044620091699802778881987206405244817066758391926795877') * z**15

coeffs = expr_k0.list()  # This gives all coefficients up to the current precision
for i, c in enumerate(coeffs):
    if c != 0:
        print("f64::from_bits(" + double_to_hex(c) + "),")
        # print(f"z^{i}: {c}")

expr_k0_den = R('1') + R('6.6872633185311663100009203016549852528062449248109539') * z + R('10.068311487338696570818625050453714367868312870255339') * z**2
    
print("expr_k0_den")
coeffs = expr_k0_den.list()  # This gives all coefficients up to the current precision
for i, c in enumerate(coeffs):
    if c != 0:
        print("f64::from_bits(" + double_to_hex(c) + "),")
        # print(f"z^{i}: {c}")

f64::from_bits(0x3ff40d931ff626ed),
f64::from_bits(0x402072fbcec226c2),
f64::from_bits(0x40275186037f1723),
f64::from_bits(0xbff146ffff2cc0e5),
f64::from_bits(0x3fda7e3ca23841ac),
f64::from_bits(0xbfd13462ad384f99),
f64::from_bits(0x3fcd3539bf027d79),
f64::from_bits(0xbfcc06b166c3fcd9),
f64::from_bits(0x3fcb5b5a47741483),
f64::from_bits(0xbfc8fa3c2bc30d81),
f64::from_bits(0x3fc3e6b86fa6ba6c),
f64::from_bits(0xbfba07af249e75b5),
f64::from_bits(0x3faa4296b62b2eb2),
f64::from_bits(0xbf92e8497eaaf439),
f64::from_bits(0x3f71247eb3aab51a),
f64::from_bits(0xbf3d3e0258904b11),
expr_k0_den
f64::from_bits(0x3ff0000000000000),
f64::from_bits(0x401abfc1f49353c1),
f64::from_bits(0x402422f9b9281f24),


In [164]:
Z = PowerSeriesRing(RealField(300), 'z', default_prec = 300)
R = RealField(170)
z = Z.gen()

from mpmath import *
mp.dps = 60;

euler_gamma = R(euler)

lg  = R(2).log()
lg4 = R(4).log()
lg64 = R(64).log()
lg4096 = R(4096).log()

expr2 = -euler_gamma + lg + R(1/4) *  z**2 *(1 - euler_gamma + lg) + (\
 z**12 *(R(49) - R(20) * euler_gamma + R(20) *lg))/R('42467328000') + (\
 z**10 *(R(137) - R(60) *euler_gamma + R(60) *lg))/R('884736000') + (\
 z**14 *(R(363) - R(140)* euler_gamma + R(140) *lg))/R('58265174016000') + (\
 z**16 *(R(761) - R(280) *euler_gamma + R(280) *lg))/R('29831769096192000') + (\
 z**18 *(R(7129) - R(2520) *euler_gamma + R(2520) *lg))/R('86989438684495872000') +\
 R(1/128)* z**4 *(R(3) - R(2) * euler_gamma + lg4) + (\
 z**6 *(R(11) - R(6) * euler_gamma + R(64).log()))/R(13824) + (\
 z**8 *(R(25) - R(12) * euler_gamma + R(4096).log()))/R(1769472)

coeffs = expr2.list()  # This gives all coefficients up to the current precision
for i, c in enumerate(coeffs):
    if c != 0:
        print_double_double("", c)
        # print("f64::from_bits(" + double_to_hex(c) + "),")
        # print(f"z^{i}: {c}")1.25331413731550025097574185194058307743736824945827587299663 + 

expr3 = R('1.25331413731550025121927282596719844509479202793343596081659') +\
R('18.1818090161526720433056569747515378514630065641474625981014') * z +\
R('84.0855308746947292990799286725009214888553349291444140660290') * z**2 +\
R('132.162365847816287381044941763470816630213463932078033240530') * z**3 +\
R('31.9226828676807644427339830797111472671372253510654795457144') * z**4 -\
R('6.55623468795551166985522946155773048406824737626044144871123') * z**5 +\
R('3.64813675168036216906333568031312648295984485156865732661545') * z**6 -\
R('3.09030769637907344287942414087144964418293628150871931378425') * z**7 +\
R('3.33386714181870765265684414717381480790589837093132038622747') * z**8 -\
R('4.15687509263144029996675634899519359876131330016912373219164') * z**9 +\
R('5.60168914866900308748492484302032849837652046070612403967421') * z**10 -\
R('7.74291668942803388088083086973577048045063529950220281515214') * z**11 +\
R('10.5182262725991203709271656214076983768897923475996777051390') * z**12 -\
R('13.5518704730239628983580812834313096171287792612392021785201') * z**13 +\
R('16.0756083151279354000726821130423204859625333785087750230562') * z**14 -\
R('17.1182809371435152577732819586801963660470433312228530729236') * z**15 +\
R('16.0011674817017251371454409588419263278595693541151351864594') * z**16 -\
R('12.8547201574813259441920569388886787474288736479192357179690') * z**17 +\
R('8.68486423302827636350417719675952558710622021096282923741362') * z**18 -\
R('4.81495629672548049328728197240253790905840957706216081678396') * z**19 +\
R('2.12449990606466172694756924925308136959633096070223397071382') * z**20 -\
R('0.715212551284256837854636761671218490310899339423567201757157') * z**21 +\
R('0.172124045885696320404925832426986371003557469307869009460024') * z**22 -\
R('0.0263211214093518652429028546952441524255064789438934805575092') * z**23 +\
R('0.00191881798965983631900602829845520749528121908949103217489718') * z**24
print("k1 asympt")

coeffs = expr3.list()  # This gives all coefficients up to the current precision
for i, c in enumerate(coeffs):
    if c != 0:
        print_dyadic(c)
        # print(c)
        # print_double_double("", c)
        # print("f64::from_bits(" + double_to_hex(c) + "),")

expr4 = R('1') + R('14.1319847014545522182859614183872365899195687422959996226761') * z +\
R('61.9082401087861097277037880035610946116051551544124209371854') * z**2 +\
R('83.7882740830616503293819459356854835383521002198942068063204') * z**3
print("k1 asympt den")

coeffs = expr4.list()  # This gives all coefficients up to the current precision
for i, c in enumerate(coeffs):
    if c != 0:
        print_dyadic(c)
        # print_double_double("", c)
        # print(c)
        # print("f64::from_bits(" + double_to_hex(c) + "),")

(0x3c1be095d05c0a81, 0x3fbdadb014541eb2),
(0x3c6037c12ba0b815, 0x3fd1dadb014541eb),
(0x3c2037c12ba0b815, 0x3f99dadb014541eb),
(0xbbe07eec845045e4, 0x3f4bb90e85debf56),
(0x3b830f4c5f3df300, 0x3eef4747696cf839),
(0x3b252fcaeee73fd1, 0x3e85d6b13b0d88ca),
(0xbab3127e7a5114b0, 0x3e14c2b6e8177e1a),
(0x3a309e5ad5685b51, 0x3d9ca0246d234e72),
(0xb9a183e5b5dac36d, 0x3d1df24eb119a2f9),
(0xb93ad95e64dfe5fc, 0x3c9896d55d330a18),
k1 asympt
DyadicFloat128 {
    sign: DyadicSign::Pos,
    exponent: -127,
    mantissa: 0xa06c98ff_b1382cb2_d9370160_5e6ea89f_u128,
},
DyadicFloat128 {
    sign: DyadicSign::Pos,
    exponent: -123,
    mantissa: 0x91745849_13f30526_b9684bb8_b82db634_u128,
},
DyadicFloat128 {
    sign: DyadicSign::Pos,
    exponent: -121,
    mantissa: 0xa82bcab3_eb3969ee_a0ffd314_025498c0_u128,
},
DyadicFloat128 {
    sign: DyadicSign::Pos,
    exponent: -120,
    mantissa: 0x842990ce_e65bb768_2cdb5294_33e86bed_u128,
},
DyadicFloat128 {
    sign: DyadicSign::Pos,
    exponent: -123,
    ma